## Overview
This notebook contains the code for the paper "<>" by <>. The paper is available at <>.

DDPMs stands for Denoising Diffusion Probabilistic Models and also knowne as diffusionmodels, score-based generative models or simply [autoencoders](https://benanne.github.io/2022/01/31/diffusion.html) as researchers have been able to achieve remarkable results with them for (un)conditional image/ausio/video generation. For example: GLIDE, DALL-E2, Latent Diffusion and ImageGen. This article employ the discrete-time (latent variable model) perspective.

## What is diffusion model?

A (denoising) diffusion model is not that complex if you compare it to other generatve models such as [Normalizing](https://aisuko.gitbook.io/wiki/ai-techniques/framework/ml_training_components#normalization) Flows, GANs or [VAEs](https://aisuko.gitbook.io/wiki/ai-techniques/stable-diffusion/vae):they all convert noise from some simple distribution to a data sample. This is also the case here where **a neural network leanrs to gradually denoise data** startubg from pure noise.

In a bit more detail for images, the  set-up consists of 2 process:

* a fixed (or predefined) forward diffusion process q of our choosing, that gradually adds Gaussian noise to an image, until you end up with pure noise.
* a learned reverse denoising process p0, where a neural network is trained to gradually denoise an image starting from pure noise, until you end up with an actual image.

More example, please see [here](https://aisuko.gitbook.io/wiki/ai-techniques/stable-diffusion/diffusion-in-image)

![](https://huggingface.co/blog/assets/78_annotated-diffusion/diffusion_figure.png)

According to the picture above, both the forward and reverse process indexed by t happen for some number of finite time steps T (the DDPM authors use T=1000). You start with t=0 where you sample a real image x0 from your data distribution, and the forward process samples some noise from a Gaussian distribution at each time step t, which is added to the image of the previous time step. Given a sufficientlt large T and a well behaved schedule for adding noise at each time step, you end up with what is called an isotropic Gaussian distribution at t =T via a gradual process.

## In more mathematical form

We need a tractable loss function whcih our neural network needs to optimize. Let q(x0) be the real data distribution, say of "real images". We can sample from this distribution to get an images, x0~q(x0). We define the forward diffusion process q(Xt|Xt-1) which adds Gaussian noise at each time step t, according to a known variance schedule
$$0 < \beta_1 < \beta_2 < ... < \beta_T < 1\\ q(\mathbf{x}_t | \mathbf{x}_{t-1}) = \mathcal{N}(\mathbf{x}_t; \sqrt{1 - \beta_t} \mathbf{x}_{t-1}, \beta_t \mathbf{I}).$$

### Gaussian distribution

Recall that a normal distribution (also called Gaussian distribution) is defined by 2 paramters:
* a mean $\\\mu$
* a variance parametrized by $\sigma^2>=0$

### Conditional Gaussian Distribution

Basically, each new (slightly noisier) image at time step t is drawn from a **conditional Gaussian distribution** with mean 
$$\mathbf{\mu}_t=\sqrt{1 - \beta_t} \mathbf{x}_{t-1}$$
and
$$\mathbf{\sigma}_t^2=\beta_t$$
which we can do by sampling
$$\mathbf{\epsilon} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$$
and then setting
$$\mathbf{x}_t = \sqrt{1-\beta_t} \mathbf{x}_{t-1} + \sqrt{\beta_t} \mathbf{\epsilon}$$


### Variance schedule
Note that the $$\mathbf{\beta_t}$$ aren't constant ar each time step t(hence the subscript) in fact one defines a so-called "vaeiance schedule" which can be linear, costine, etc. So, starting from X0, we end up with X1, X2, ..., XT, where XT is pure Gaussian noise if we set the schedule appropriately.

### Solving the problem in reverse
Now, if we knew the conditional distrbution
$$p(\mathbf{x}_{t-1}|\mathbf{x}_t)$$
then we could fun the process in reverse: by sampling some random Gaussian noise XT, and the gradually "denoise" it so that were end up with a sample from the real distribution X0.

However, we don't know $$p(\mathbf{x}_{t-1}|\mathbf{x}_t)$$. It is intractable since it requires knowing the distribution of all possible images in order ro calculate the conditional probability. Hence, we are going to leverage a neural network to **approximate (learn) this conditional probability distribution**, let's call it $$p(\mathbf{x}_{t-1}|\mathbf{x}_t)$$, with parameters $$\theta$$, which being the parameters of the neural network, updated via gradient descent.

### Building the noise predictor (neural network/ loss function)

We need a neural network to represent a (conditional) probability distribution of the backward process. If we assume this reverse process is Gaussian as well, then recall the any Guassian distribution is defined by 2 parameters:

* a mean parameterized by $$\mu_\theta$$
* a variance parameterized by $$\Sigma_\theta$$

### Parameterizing the process

Formula below which the mean and variance are also consitioned on the noise level t.

$$ p_\theta (\mathbf{x}_{t-1} | \mathbf{x}_t) = \mathcal{N}(\mathbf{x}_{t-1}; \mu_\theta(\mathbf{x}_{t},t), \Sigma_\theta (\mathbf{x}_{t},t))$$

### NN learn/represent the mean and variance

This is not shown in the paper, it was then later improved in other papers, where a neural network also learns the variance of this backwards process, bedised the mean.

## Defining an objective funciton (by reparametrizing the mean)

To derive an objective function to learn the mean of the backward process in the paper is the combination of q and p0, it can be seen as a variational VAE(Kingma et al., 2013). Hence the **variatinal lower bound** (ELBO) can be used to minimize the negative log-likelihood with respect to ground truth data sample X0. It turns out that the ELBO for this process is a sum of losses at each time step t, L=L0+L1+...+LT.
By construction of the forward q process and backward process, each term (except for l0) of the loss is actually the **KL divergence between 2 Gaussioan distributions**, which can be written explicitly as an L2-loss with repect to the means.

A direct consuquence of the constructed forward process q, as shown by Sohl-Dickstein et al., is that we can samoke Xt at any arbirtrary noise level cinditioned on X0 (since sums of Gaussians is also Gaussian). This is very convenient: we don't need to apply q repeatedly in order to sample Xt. We have that:
$$q(\mathbf{x}_t | \mathbf{x}_0) = \cal{N}(\mathbf{x}_t; \sqrt{\bar{\alpha}_t} \mathbf{x}_0, (1- \bar{\alpha}_t) \mathbf{I})$$
with $$\alpha_t := 1 - \beta_t$$ and $$\bar{\alpha}_t := \Pi_{s=1}^{t} \alpha_s$$

Let's refer to this equation as the "nice property". This means we can sample Gaussian noise and scale it appropriatly and add it to \\(\mathbf{x}_0\\) to get \\(\mathbf{x}_t\\) directly. Note that the \\(\bar{\alpha}_t\\) are functions of the known \\(\beta_t\\) variance schedule and thus are also known and can be precomputed. This then allows us, during training, to **optimize random terms of the loss function \\(L\\)** (or in other words, to randomly sample \\(t\\) during training and optimize \\(L_t\\)).

Another beauty of this property, as shown in th paper. is that one can instead **reparametrize the mean to make the neural network learn (predict) the added noise (via a network \\(\mathbf{\epsilon}_\theta(\mathbf{x}_t, t)\\)) for noise level \\(t\\)** in the KL terms which constitute the losses. This means that our **neural network becomes a noise predictor, rather than a (direct) mean predictor**. The mean can be computed as follows:

$$ \mathbf{\mu}_\theta(\mathbf{x}_t, t) = \frac{1}{\sqrt{\alpha_t}} \left(  \mathbf{x}_t - \frac{\beta_t}{\sqrt{1- \bar{\alpha}_t}} \mathbf{\epsilon}_\theta(\mathbf{x}_t, t) \right)$$

The final objective function Lt then looks as follows:
$$ \| \mathbf{\epsilon} - \mathbf{\epsilon}_\theta(\mathbf{x}_t, t) \|^2 = \| \mathbf{\epsilon} - \mathbf{\epsilon}_\theta( \sqrt{\bar{\alpha}_t} \mathbf{x}_0 + \sqrt{(1- \bar{\alpha}_t)  } \mathbf{\epsilon}, t) \|^2.$$


### Training algorithm

Here, X0 is the inital (real, uncorrupted) image, and we see the direct noise level t sample given by the fixed forward process. \\(\mathbf{\epsilon}\\) is the pure noise sampled at time step t, and \\(\mathbf{\epsilon}_\theta (\mathbf{x}_t, t)\\) is our nerual network. The neural network is optimized using a simple mean squared error (MSE) between the true and the predicted Gaussian noise.

<p align="center">
    <img src="https://huggingface.co/blog/assets/78_annotated-diffusion/training.png" width="500" />
</p>

According to the picture above:
* we take a random sample Xo from the real unknown and possibily complex data distribution q(Xo)
* we sample a noise level t uniformally between 1 and T(i.e: a random time step)
* we sample some noise from a Guassian distribution and corrupt the input by this noise at level t(using the nice property defined above)
* the neural network is trained to predict this noise based on the corrupted image Xt(i.e. noise applied on X0 based on known schedule \(\beta_t\\)

In reality, all of this is done on batches of data, as one uses stochastic gradient descent to optimize the neural network parameters.

## The neural network

The neural network needs to take in a noised image at a particular time step and return the predicted noise. Note that the predicted noise is a tensor that has the same size/resolution as the input image. So, technically, the network takes in and outputs tensors of the same shape. What type of neural network can we use for this?

What is typically used here is very similar to that of an [Autoencoder](https://aisuko.gitbook.io/wiki/ai-techniques/stable-diffusion/vae#variational-autoencoder), which you may remember from typical "intro to deep learning" tutorials.

In terms of architecture, the paper went for a U-Net, introduced by ([Ronneberger et al., 2015](https://arxiv.org/abs/1505.04597)) (which at the time, achived state-of-the-art results for medical image segmentation). This network, like any autoencoder, consists of a bottleneck in the middle that makes sure the network learns only the most important information. Importantly, it introduced **redisual connections** between the encder and decoder, greatly improving gradient flow(inspired by ResNet in He et al.,2015).

<p align="center">
    <img src="https://huggingface.co/blog/assets/78_annotated-diffusion/unet_architecture.jpg" width="500" />
</p>

As can be seen, a U-Net model first downsamples the input (i.e. makes the input smaller in terms of spatial resolution), after which upsamling is performed.

## Let's implement this network

### Preparation
Installing the required packages and importing the libraries. `einops` is flexsible and poweful tensor operations tool, [more details.](https://github.com/arogozhnikov/einops)

In [ ]:
# For the Kaggle environment
pip install pytorch=2.0.1 einops==0.6.1 datasets=2.13.1 matplotlib==3.7.1 tqdm==4.65.0

In [ ]:
from __future__ import annotations
import math
from inspect import isfunction
from functools import partial

#  %matplotlib inline is a magic command in Python that is used to display Matplotlib plots directly in the Jupyter Notebook or IPython shell
%matplotlib inline 
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from einops import rearrange, reduce
from einops.layers.torch import Rearrange

import torch
from torch import nn, einsum
import torch.nn.functional as F

### Network helpers

First, we define some helper functions and classes which will be used when implementing the neural network. Importantly, we define a `Residual` module, which simply adds the input to the output of a particular function(in other words, **adds a residual connection to a particular function**).

We also define aliases for the up and downsampling operations.

In [ ]:
def exists(val):
    return val is not None

def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d

def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder >0:
        arr.append(remainder)
    return arr

def Upsample(dim, dim_out=None):
    return nn.Sequential(
        nn.Upsample(scale_factor=2, mode="nearest"),
        nn.Conv2d(dim, default(dim_out,dim), 3, padding=1),
    )

def Downsample(dim, dim_out=None):
    # No More Strided Convolutions or Pooling
    return nn.Sequential(
        Rearrange("b c (h p1) (w p2) -> b (c p1 p2) h w", p1=2, p2=2),
        nn.Conv2d(dim*4, default(dim_out, dim), 1),
    )

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, *args, **kwargs):
        return self.fn(x, *args, **kwargs) + x

### Position embeddings

As the parameters of the neutral network are shared across time (noise level), the authors employ sinusoidal position embeddings to encode t, inspired by the Transformer [(Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762). **This makes the neural network "know" at which particular time step (noise level) it is operating, for every time in a batch.**

The [SinusoidalPostionEmbeddings](#TODO) module takes a tensor of shape (batch_size, 1) as input (i.e. the noise levels of several noisy images in a batch), and turns this into a tensor of shape (batch_size, dim), with dim being the dimensonality of the position embeddings. This is then added to each residual block, as we will see further.

In [ ]:
class SinusoidalPostionEmbeddings(nn.Module):
    
    def __init__(self, dim) -> None:
        super().__init__()
        self.dim = dim
    
    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

### ResNet block

Next, we deifne the core building block of the U-Net model. The paper employed a Wide ResNet block [(Zagoruyko and Komodakis, 2016)](https://arxiv.org/abs/1605.07146), which is a ResNet block with a larger number of channels, but the standard convolutional layer was replaced by a "weight standardized" version by other people, which works better in combination with group normalization. More detail see [Kolesnikov et al,. 2019](https://arxiv.org/abs/1912.11370).

In [ ]:
class WeightStandardizedConv2d(nn.Conv2d):
    """
    https://arxiv.org/abs/1903.10520
    weight standardization purportedly works synergistically with group normalization
    """

    def forward(self, x):
        eps = 1e-5 if x.dtype is torch.float32 else 1e-3

        weight = self.weight
        mean = reduce(weight, "o ... -> o 1 1 1", "mean") 
        var = reduce(weight, "o ... -> o 1 1 1", partial(torch.var, unbiased=False))
        normalized_weight = (weight - mean) / (var + eps).rsqrt()

        return F.conv2d(x, normalized_weight, self.bias, self.stride, self.padding, self.dilation, self.groups,)

class Block(nn.Module):
    def __init__(self, dim, dim_out, groups=8) -> None:
        super().__init__()
        self.proj = WeightStandardizedConv2d(dim, dim_out, 3, padding=1)
        self.norm = nn.GroupNorm(num_groups=groups, num_channels=dim_out)
        self.act = nn.SiLU()
    
    def forward(self, x, scale_shift=None):
        x = self.proj(x)
        x = self.norm(x)

        if exists(scale_shift):
            scale, shift = scale_shift
            x = x * (scale+1) + shift

        x = self.act(x)
        return x

class ResnetBlock(nn.Module):
    """
    https://arxiv.org/abs/1512.03385
    """

    def __init__(self, dim, dim_out, *, time_emb_dim=None, groups=8) -> None:
        super().__init__()
        self.mlp=(
            nn.Sequential(nn.SiLU(), nn.Linear(time_emb_dim, dim_out*2))
            if exists(time_emb_dim)
            else None
        )

        self.block1 = Block(dim, dim_out, groups=groups)
        self.block2 = Block(dim_out, dim_out, groups=groups)
        self.res_conv = nn.Conv2d(dim, dim_out, 1) if dim != dim_out else nn.Identity()
    
    def forward(self, x, time_emb=None):
        scale_shift = None
        if exists(self.mlp) and exists(time_emb):
            time_emb =self.mlp(time_emb)
            time_emb == rearrange(time_emb, "b c -> b c 1 1")
            scale_shift = time_emb.chunk(2, dim=1)
        
        h =self.block1(x, scale_shift=scale_shift)
        h =self.block2(h)
        return h + self.res_conv(x)

### Attention module

Next, we define the attention module, which the paper added in between the convolutional blocks. Attention is the building block of the famous Transformer architecture [(Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762), which is used in NLP for example. The attention module is a bit more complex than the ResNet block, but it is still quite simple. It takes in a tensor of shape (batch_size, channels, height, width) and returns a tensor of the same shape. The attention module is used to capture long-range dependencies in the image, which is important for the diffusion process. Here are two variants of attention:
* One is regular multi-head self-attention (as used in the Transformer)
* The other one is a linear attetion variant [(Shen et al.,2018)](https://arxiv.org/abs/1812.01243), whoise time-and memory requirements scale linear in the sequence length, as opposed to quadratic for the regular attention.

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, heads=4, dim_head=32) -> None:
        super().__init__()
        self.scale = dim_head ** -0.5
        self.heads=heads
        hidden_dim = dim_head * heads
        self.to_qkv = nn.Conv2d(dim, hidden_dim*3, 1, bias=False)
        self.to_out = nn.Conv2d(hidden_dim, dim, 1)
    
    def forward(self, x):
        b, c, h, w = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=1)
        q, k,v = map(
            lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
        )
        q = q * self.scale

        sim = einsum("b h d i, b h d j -> b h i j", q, k)
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        out = einsum("b h i j, b h d j -> b h i d", attn, v)
        out = rearrange(out, "b h (x y) d -> b (h d) x y", x=h, y=w)
        return self.to_out(out)

class LinearAttention(nn.Module):
    def __init__(self, dim, heads=4, dim_head=32) -> None:
        super().__init__()
        self.scale = dim_head ** -0.5
        self.heads=heads
        hidden_dim = dim_head * heads
        self.to_qkv = nn.Linear(dim, hidden_dim*3, 1, bias=False)
        self.to_out = nn.Sequential(
            nn.Conv2d(hidden_dim, dim, 1),
            nn.GroupNorm(1, dim))
        
    def forward(self, x):
        b, c, h ,w = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=1)
        q, k,v = map(
            lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
        )

        q = q.softmax(dim=-2)
        k = k.softmax(dim=-1)

        q = q * self.scale
        context = torch.einsum("b h d n, b h e n -> b h d e", k, v)

        out = torch.einsum("b h d e, b h d n -> b h e n", context, q)
        out = rearrange(out, "b h c (x y) -> b (h c) x y", h=self.heads, x=h, y=w)
        return self.to_out(out)

### Group normalization

The paper used group normalization [(Wu and He, 2018)](https://arxiv.org/abs/1803.08494) instead of batch normalization, as it is more stable and works better in combination with weight standardization. It is very similar to batch normalization, but instead of normalizing over the batch dimension, it normalizes over the channel dimension. This is important, as the batch dimension is not always the same size, but the channel dimension is. Below, we define a PreNorm class, which will be used to apply groupnorm before the attention layer.

In [ ]:
class PreNorm(nn.Module):
    def __init__(self, dim, fn) -> None:
        super().__init__()
        self.fn = fn
        self.norm = nn.GroupNorm(1, dim)

    
    def forward(self, x):
        x = self.norm(x)
        return self.fn(x)

### Conditional U-Net

Now that we've defined all building blocks ([position embeddings](#TODO), [ResNet blocks](#TODO), [attention](#TODO) and [group normalization](#TODO)), it's time to define the entire neural network. Recall that the job of the network ϵθ(Xt,t) is to take in a batch of noisy images and their respective noise levels, and output the noise added to the input. More formally:

* the network takes a batch of noisy images of shape (batch_size, num_channels, height, width) and a batch of noise levels of shape (batch_size, 1) as input, and returns a tensor of shape (batch_size, num_channels, height, width).

The network is built up as follows:
* first, a convolutional layer is applied to the batch of noisy images, and position embeddings are computed for the noise levels
* next, a sequence of downsampling stages are applied. Each downsampling stage consists of 2 ResNet blocks + groupnorm + attention + residual connection + a downsample operation
* at the middle of the network, afaub ResNet blocks are applied, interleaved with attention
* next, a sequence of upsampling stages are applied. Each upsampling stage consists of 2 ResNet blocks + groupnorm + attention + residual connection + an upsample operation
* finally, a ResNet block followed by a convolutional layer is applied

Ultimately, neural networks stack up layers as if they were lego blocks.

In [ ]:
class Unet(nn.Module):
    def __init__(self, dim, init_dim=None, out_dim=None, dim_mults=(1,2,4,8), channels=3, self_condition=False, resnet_block_groups=4) -> None:
        super().__init__()

        # determine dmensions
        self.channels = channels
        self.self_condition = self_condition
        input_channels = channels * (2 if self_condition else 1)
        init_dim =default(init_dim, dim)
        self.init_conv = nn.Conv2d(input_channels, init_dim, 1, padding=0) # changed to 1 and 0 from 7,3
        dims = [init_dim, *map(lambda m: dim*m, dim_mults)]
        in_out = list(zip(dims[:-1],dims[1:]))

        block_klass = partial(ResnetBlock, groups=resnet_block_groups)

        # time embedding
        time_dim =dim*4

        self.time_mlp =nn.Sequential(
            SinusoidalPostionEmbeddings(dim),
            nn.Linear(dim, time_dim),
            nn.GELU(),
            nn.Linear(time_dim, time_dim),
        )

        # layers
        self.downs = nn.ModuleList([])
        self.ups = nn.ModuleList([])
        num_resolutions = len(in_out)

        for ind, (dim_in, dim_out) in enumerate(in_out):
            is_last = ind >= (num_resolutions -1)

            self.downs.append(
                nn.ModuleList(
                    [
                        block_klass(dim_in, dim_in, time_emb_dim=time_dim),
                        block_klass(dim_in, dim_in, time_emb_dim=time_dim),
                        Residual(PreNorm(dim_in, LinearAttention(dim_in))),
                        Downsample(dim_in, dim_out)
                        if not is_last
                        else nn.Conv2d(dim_in, dim_out, 3, padding=1),
                    ]
                )
            )

        mid_dim =dims[-1]
        self.mid_block1=block_klass(mid_dim, mid_dim, time_emb_dim=time_dim)
        self.mid_attn=Residual(PreNorm(mid_dim, Attention(mid_dim)))
        self.mid_block2=block_klass(mid_dim, mid_dim, time_emb_dim=time_dim)

        for ind, (dim_in, dim_out) in enumerate(reversed(in_out)):
            is_last = ind == (len(in_out)-1)

            self.ups.append(
                nn.ModuleList(
                    [
                        block_klass(dim_out+dim_in, dim_out, time_emb_dim=time_dim),
                        block_klass(dim_out+dim_in, dim_in, dim_out, time_emb_dim=time_dim),
                        Residual(PreNorm(dim_out, LinearAttention(dim_out))),
                        Upsample(dim_out, dim_in)
                        if not is_last
                        else nn.Conv2d(dim_out, dim_in, 3, padding=1),
                    ]
                )
            )
        
        self.out_dim =default(out_dim, channels)
        self.final_res_block = block_klass(dim*2, dim, time_emb_dim=time_dim)
        self.final_conv = nn.Conv2d(dim, self.out_dim, 1)
    

    def forward(self, x, time, x_self_cond=None):
        if self.self_condition:
            x_self_cond = default(x_self_cond, lambda: torch.zeros_like(x))
            x = torch.cat((x_self_cond, x), dim=1)
        
        x = self.init_conv(x)
        r = x.clone()

        t = self.time_mlp(time)

        h = []

        for block1, block2, attn, downsample in self.downs:
            x = block1(x, t)
            h.append(x)

            x = block2(x, t)
            x = attn(x)
            h.append(x)
            x = downsample(x)
        
        x =self.mid_block1(x, t)
        x =self.mid_attn(x)
        x =self.mid_block2(x, t)

        for block1, block2, attn, upsample in self.ups:
            x = torch.cat((x, h.pop()), dim=1)
            x = block1(x, t)

            x= torch.cat((x, h.pop()), dim=1)
            x = block2(x, t)
            x = attn(x)
            x = upsample(x)
        
        x = torch.cat((x, r), dim=1)
        x = self.final_res_block(x, t)
        return self.final_conv(x)

## Credit

* [The Annotated Diffusion Model](https://huggingface.co/blog/annotated-diffusion)